# Web Parser
- Author: Aidan Marlin
- Date: 20220630
- Version: 1.0
- Description: Jupyter Notebook to distil results from web scans

In [ ]:
target = "www.bing.com"
top_target = "bing.com"

###
### No need to modify from here
###
import socket
import os

os.environ["PATH"] = os.environ["PATH"] + ":/run/current-system/sw/bin"

target_ip = socket.gethostbyname(target) 
http_target = "http://" + target
https_target = "https://" + target

!mkdir -p "`pwd`/parsed"
!pwd

# Target

In [ ]:
print(target)
print(target_ip)


# DNS

In [ ]:
!cat "`pwd`/tools/amass/results.txt" | jq -r '.name' > "`pwd`/parsed/dns_amass"
!cat "`pwd`/tools/ffuf_subdomain/"* | grep -E "^Host: " | choose 1 > "`pwd`/parsed/dns_ffuf"
!cat "`pwd`/tools/finalrecon/"*".txt" | grep " IN " | choose 0 | sort | uniq | sed 's/\.$//g' > "`pwd`/parsed/dns_finalrecon"
!cat "`pwd`/tools/knockpy/"*".json" | grep -E '^    "' | grep -v '_meta"' | choose 0 | tr -d '":' | sort | uniq > "`pwd`/parsed/dns_knockpy"
!cat "`pwd`/tools/subfinder/results.txt" | jq -r ".host" | sort | uniq > "`pwd`/parsed/dns_subfinder"
!cat "`pwd`/tools/sudomy/"*"/"*"/subdomain.txt" "`pwd`/tools/sudomy/"*"/"*"/subdomain.txt" | sort | uniq > "`pwd`/parsed/dns_sudomy"
!cat "`pwd`/tools/puredns/"*".txt" | sort | uniq > "`pwd`/parsed/dns_puredns"

!cat "`pwd`/parsed/dns_"* | sort | uniq > "`pwd`/parsed/dns.txt"
!cat "`pwd`/parsed/dns.txt"


In [ ]:
!time docker run -v "`pwd`:/mnt" --rm secsi/massdns /mnt/parsed/dns.txt -r /mnt/tools/resolver.txt -o S -q


# httpx

In [ ]:
!cat "`pwd`/parsed/dns_"* | sort | uniq | httpx -silent -fr -ip -status-code -title -tech-detect -asn


# Hosts

## Alive

In [ ]:
!cat "`pwd`/tools/nmap/internal_ping_scan.gnmap" | grep ": Up" | choose 1 | sort | uniq


## TCP Ports

In [ ]:
!cat "`pwd`/tools/masscan/tcp_all.grep" | grep -E "^Timestamp" | choose 3 6 | sed 's#/.*##g' | sed 's/ /:/g' | sort | uniq


## UDP Ports

In [ ]:
!cat "`pwd`/tools/nmap/udp_common.gnmap" | grep "open" | sort | uniq


## TLS / SSL

In [ ]:
!cat "`pwd`/tools/testssl/"*".csv" | grep -Ev '"INFO"|"OK"|"WARN"' | choose -f '"' 0 | grep -Ev "^id$"


# URLs

## JavaScript

In [ ]:
!cat "`pwd`/tools/photon/"*"/"*".txt" | grep -i "\.js" | sed 's/\?.*//g' | sort | uniq > "`pwd`/parsed/js_photon"
!cat "`pwd`/tools/finalrecon/"*".txt" | grep -i "\.js" | sed 's/\?.*//g' | sort | uniq > "`pwd`/parsed/js_finalrecon"
!cat "`pwd`/tools/sudomy/"*"/"*"/interest/"*"-js.out" | grep -i "\.js" | sed 's/\?.*//g' | sort | uniq > "`pwd`/parsed/js_finalrecon"
!cat "`pwd`/tools/gospider/"* | grep -E "^\[javascript\]" | choose -f '- ' 1 | sed 's/\?.*//g' | sort | uniq > "`pwd`/parsed/js_gospider"
!cat "`pwd`/tools/hakrawler/results.txt" | grep -E "\[script\]" | choose 1 | grep -i "\.js" | sed 's/\?.*//g' | sort | uniq > "`pwd`/parsed/js_hakrawler"

!cat "`pwd`/parsed/js_"* | sort | uniq


## CSS

In [ ]:
!cat "`pwd`/tools/finalrecon/"*".txt" | grep -E "\.css" | sed 's/\?.*//g' | sort | uniq > "`pwd`/parsed/css_finalrecon"
!cat "`pwd`/parsed/css_"*


# Nikto Issues

In [ ]:
!cat "`pwd`/tools/nikto/nikto" | jq | grep msg | choose -f ": " 1 | tr -d '"' | sort | uniq


# Notify

In [ ]:
!/home/user/git/maxos/scripts/telegram_notify.sh -a -m "PT Web Parser complete for {target}."


# End Time

In [ ]:
!date
